In [62]:
import smr
import numpy as np
from open_ephys.analysis import Session
import lfp
import time
# import matplotlib.pyplot as plt
import pickle
import OpenEphys
import re

In [63]:
data_path = 'D:\\AwakeData\\Deep Array\\230413\\'
out_path = 'D:\\AwakeData\\Python\\230413\\continuous\\'

In [64]:
# define parameters for LFP processing
params = {
    'filter_order': 6,
    'stopband_attenuation': 50,
    'cutoff_frequency': 250,
    'downsample_frequency': 500,
    'fs': np.nan,
    'ripple': 1,
    'scale_data': False,
    'sample_per': 300000,
    'sample_count': np.nan,
    'chan_num': np.nan,
}

In [65]:
session = Session(data_path)

In [66]:
params['fs'] = session.recordnodes[0].recordings[0].continuous[0].metadata['sample_rate']

In [67]:
time_0 = time.time()
for node_index in range(len(session.recordnodes)):

    # iterate through all recordings in the current recordnode
    for rec_index in range(len(session.recordnodes[node_index].recordings)):
        
        d =session.recordnodes[0].recordings[rec_index].directory
        r_dir = d.rsplit("\\", 1)[-1]
        print(r_dir)
        
        recording = session.recordnodes[node_index].recordings[rec_index]
        params['sample_count'] = len(recording.continuous[0].sample_numbers)
        params['chan_num'] = np.array(recording.continuous[0].samples.shape[1])
        time_array = np.zeros(params['chan_num'])
        current_out_path = out_path + r_dir +'\\'
        print(current_out_path)
        for chan_index in range(params['chan_num']):
            time_array[chan_index] = time.time()
            LFP = lfp.lfp(recording.continuous[0].samples[:, chan_index], params['fs'])
            LFP.filter = lfp.filter(params)
            LFP.filter.cheby2_lowpass(LFP.Y)
            LFP.subsample(params['downsample_frequency'])
            chan_string = str(chan_index)
            file_name = f"{current_out_path}channel_{chan_index}.pkl"
            breakpoint()
            # save data to disk
            with open(file_name, 'wb') as f:
                pickle.dump(LFP, f)

            elapsed_time_seconds = time_array[chan_index] - time_0
        print(f"The elapsed time in seconds is {elapsed_time_seconds}")

recording1
D:\AwakeData\Python\230413\continuous\recording1\


KeyboardInterrupt: 

In [ ]:
# open LFP file 
file_path = 'D:\\AwakeData\\Python\\230413\\continuous\\recording4\\channel_0.pkl'
with open(file_path, 'rb') as file:
    lfp_data = pickle.load(file)

In [ ]:
# load recording parameters
rec_par_file = 'C:\\Henry\\CurrentAwakeData\\Phy\\deep_array\\230413\\recording_4\\params.py'
recording_params = OpenEphys.get_recording_params(rec_par_file)

# get spike times from kilosort
spike_times = np.load('C:\\Henry\\CurrentAwakeData\\Phy\\deep_array\\230413\\recording_4\\spike_times.npy')
spike_times = spike_times / recording_params['sample_rate']
spike_clusters = np.load('C:\\Henry\\CurrentAwakeData\\Phy\\deep_array\\230413\\recording_4\\spike_clusters.npy')



In [ ]:
# load TTL triggers from

event_dir = 'D:\\AwakeData\\Deep Array\\230413\\Record Node 101\\experiment1\\recording4\\events\\Acquisition_Board-100.Rhythm Data\\TTL\\'
ttl_times_file = 'timestamps.npy'

ttl_ts = np.load(event_dir + ttl_times_file)



In [ ]:
ttl_ts

In [ ]:
ttl_ts[0]

In [ ]:
165697280/30000

In [ ]:
# load sync message
synch_file = 'D:\\AwakeData\\Deep Array\\230413\\Record Node 101\\experiment1\\recording4\\sync_messages.txt'


with open(synch_file , 'r') as file:
    synch_message = file.read()

In [ ]:
pattern = r"Start Time for Acquisition Board \(100\) - Rhythm Data @ (\d+) Hz: (\d+)"

match = re.search(pattern, synch_message)
if match:
    Fs = np.int64(match.group(1))
    acqu_start_time = np.int64(match.group(2))
    acqu_start_time = acqu_start_time/ Fs
    print("Frequency:", Fs)  # Output: 555
    print("Start Time:", acqu_start_time)  # Output: 165697280

In [ ]:
np.int64(Fs)

In [ ]:
smr_dir_name = 'D:\\AwakeData\\Deep Array\\230413\\Record Node 101\\experiment1\\recording4\\smr_triggers\\'
smr_file_name = 'oe_tmj_230413_ori_003.csv'
smr_file_name = smr_dir_name + smr_file_name
print(smr_file_name)
smr_triggers = smr.load_smr_triggers(smr_file_name)

In [ ]:
valid_trials = smr_triggers.IV>-1

smr_triggers.loc[valid_trials, 'TrialStart'] -= acqu_start_time



In [ ]:
smr_triggers.TrialStart[valid_trials]

In [ ]:
trial_start =smr_triggers.TrialStart[valid_trials] - acqu_start_time

In [ ]:
smr_triggers.loc